In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import requests
import time
import urllib
from tqdm import tqdm
import sys
import os

import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import re
import datetime
import json
from IPython.display import display

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException # 검색 결과가 없는 경우 예외 처리
from lxml.html import fromstring
from bs4 import BeautifulSoup

In [2]:
# 수집 날짜 지정(전일자)
before_one_day = datetime.datetime.now() - datetime.timedelta(days=1)
D, db_date = before_one_day.strftime("%Y.%m.%d"), before_one_day.strftime("%y%m%d")

In [5]:
conn = db_connection('')
news_link = pd.read_sql(f"select * from table_name where text IS NULL;", conn)
conn.close()

In [ ]:
news_link = pd.read_excel(f'news_href_{db_date}.xlsx')

In [6]:
print(len(news_link))
news_link.head()

5155


,id,news,title,href,text,write_date,regist_date
0,1,TV리포트,"20년 미스터리 풀렸다.. 새벽2시 日서 사라진 김국진 ""강수지 만났다""('갓경규')",https://entertain.naver.com/now/read?oid=213&a...,None,None,2023-11-02 06:25:06
1,2,일간스포츠,[TVis] 딘딘 “’세상에 이런 일이’ MC 예상 못해…’포스트 임성훈’ 고려한 ...,https://entertain.naver.com/now/read?oid=241&a...,None,None,2023-11-02 06:25:06
2,3,스포츠조선,"""계속 토익강사하지"" 17기 옥순, 영철과 대화 중 눈물 ""연애 기회 없어"" ('나...",https://entertain.naver.com/now/read?oid=076&a...,None,None,2023-11-02 06:25:06
3,4,OSEN,'고딩엄빠4' 극단적 선택 시도→극적 화해···엄마+딸 18세 임신 사연[종합],https://entertain.naver.com/now/read?oid=109&a...,None,None,2023-11-02 06:25:06
4,5,마이데일리,"열연이 섬세하다…전종서, '발레리나'로 도약했다",https://entertain.naver.com/now/read?oid=117&a...,None,None,2023-11-02 06:25:06


---

In [18]:
# 뉴스 내용 담을 DataFrame
news_text = pd.DataFrame(columns = ['title', 'text', 'write_date'])

drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for i in tqdm(range(len(news_link[:500]))):
    drv.get(news_link['href'][i])
    time.sleep(1.5)
    soup = BeautifulSoup(drv.page_source, 'html.parser')
    
    title = soup.find('h2', {'class':'end_tit'}).get_text().replace('\n', '').replace('\t', '')

    date = soup.find('div', {'class':'article_info'}).find('span', {'class':'author'}).get_text()
    date = date.split('기사입력')[-1].split(' ')[0][:-1].replace('.', '-')

    try:
        del_w = soup.find('div', {'id':'articeBody'}).find('em', {'class':'img_desc'}).get_text()
        text = soup.find('div', {'id':'articeBody'}).get_text()
        text = text.replace('\n', ' ').split(del_w)[-1].replace('\'', '').replace('\t', '').replace('"', '').replace.strip()

        if len(soup.find('div', {'id':'articeBody'}).get_text().replace('\n', ' ').split(del_w))>2:
            text = soup.find('div', {'id':'articeBody'}).get_text()
            text = text.replace('\n', ' ').replace('\'', '').replace('\t', '').replace('"', '').strip()      
    except:
        text = soup.find('div', {'id':'articeBody'}).get_text()
        text = text.replace('\n', ' ').replace('\'', '').replace('\t', '').replace('"', '').strip()

    data = pd.DataFrame({'id':news_link['id'][i],'title':[title], 'write_date':[date], 'text':[text]})
    news_text = pd.concat([news_text, data], ignore_index=True)

drv.close()

100%|█████████████████████████████████████████| 500/500 [17:42<00:00,  2.13s/it]


In [21]:
news_text

,title,text,write_date,id
0,"20년 미스터리 풀렸다.. 새벽2시 日서 사라진 김국진 ""강수지 만났다""('갓경규')","20년 미스터리가 풀린 순간, 이경규는 전율하고 김국진은 웃었다. 무슨 사연일까.1...",2023-11-01,1.0
1,[TVis] 딘딘 “’세상에 이런 일이’ MC 예상 못해…’포스트 임성훈’ 고려한 ...,사진제공=MBC 캡처 가수 딘딘이 ‘세상에 이런 일이’ MC를 맡은 소감을 밝혔다....,2023-11-01,2.0
2,"""계속 토익강사하지"" 17기 옥순, 영철과 대화 중 눈물 ""연애 기회 없어"" ('나...",[스포츠조선닷컴 이우주 기자] 나는 솔로 옥순이 복합적인 자신의 상황에 결국 눈물을...,2023-11-01,3.0
3,'고딩엄빠4' 극단적 선택 시도→극적 화해···엄마+딸 18세 임신 사연[종합],[OSEN=오세진 기자] MBN ‘어른들은 모르는 고딩엄빠4’(이하 ‘고딩엄빠4’)...,2023-11-01,4.0
4,"열연이 섬세하다…전종서, '발레리나'로 도약했다",배우 전종서 / 넷플릭스 배우 전종서 / 넷플릭스 배우 전종서 / 넷플릭스[마이데일...,2023-11-01,5.0
...,...,...,...,...
495,"[포토] 김성령, 빛나는 미소로",배우 김성령. <강영국 기자>제10회 이데일리 문화대상 레드카펫 행사가 1일 오후 ...,2023-11-01,496.0
496,"[TD영상] 위클리, 라치카와 함께한 신곡 '칭찬만 받은 연습벌레 위클리'",Loading... player_likesound_mutesound_mutesoun...,2023-11-01,497.0
497,"[포토] 김성령, 모델같은 워킹으로",배우 김성령. <강영국 기자>제10회 이데일리 문화대상 레드카펫 행사가 1일 오후 ...,2023-11-01,498.0
498,"김선호→김슬기, 대학로 뜬다..연극 '행복을 찾아서' 피케팅 예고",[OSEN=박소영 기자] 배우 김선호가 다시 한번 연극 무대에 선다.연극 행복을 찾...,2023-11-01,499.0


In [ ]:
news_text.to_excel(f'news_href_{db_date}.xlsx', index=False)

In [30]:
for i in tqdm(range(len(news_text))):
    conn = db_connection('')
    cur = conn.cursor()

    cur.execute(
        f'update table_name set text="{news_text["text"][i]}", write_date="{news_text["write_date"][i]}" where id = "{news_text["id"][i]}"'
    )
    conn.commit()

    cur.close()
    conn.close()

100%|█████████████████████████████████████████| 500/500 [00:32<00:00, 15.23it/s]
